## Assigning children to sandwiches

In [2]:
using JuMP, Clp, NamedArrays

# create the index sets (sandwich types, children)
sandwiches = [ :PBJ, :Turkey, :Ham, :Veggie, :Tuna ]
kids = [ :Akhil, :Brianna, :Carlos, :Dinah, :Edward ]

# create the input matrix for a NamedArray that is indexed by sandwiches (rows)
# and children (columns) with elements showing the preferences
prefs = [ 2 9 1 9 6
        9 0 5 5 8
        7 0 10 2 6
        0 9 8 1 7
        4 1 4 0 4]

# create a NamedArray showing each child's sandwich preferences
preferences = NamedArray( prefs, (sandwiches,kids), ("sandwich","kid"))

m = Model(Clp.Optimizer)

# variable for assigning each sandwich to a child
# NOTE: This is just a normal variable! We don't need to specify "binary"
# in this case. We'll see why soon.
@variable(m, x[sandwiches,kids] >= 0)

# each kid gets exactly 1 sandwich
@constraint(m, a[j in kids], sum(x[i,j] for i in sandwiches) == 1 ) 
# 각 사람 별로 NamedArray의 column 별로 더한 것.

# each sandwich gets assigned to one kid
@constraint(m, b[i in sandwiches], sum(x[i,j] for j in kids) == 1 )
# 각 샌드위치 별로 NamedArray의 row 별로 더한 것.

# maximize overall preference
@objective(m, Max, sum( x[i,j]*preferences[i,j] for i in sandwiches, j in kids ) )

optimize!(m)

# we can print the solution to a NamedArray for ease of understanding
assignment = NamedArray( [ (value(x[i,j])) for i in sandwiches, j in kids ], (sandwiches, kids), ("sandwich","kid"))

Coin0506I Presolve 10 (0) rows, 25 (0) columns and 50 (0) elements
Clp0006I 0  Obj 0 Primal inf 9.999999 (10) Dual inf 117 (21)
Clp0006I 7  Obj 41
Clp0000I Optimal - objective value 41
Clp0032I Optimal objective 41 - 7 iterations time 0.002


5×5 Named Matrix{Float64}
sandwich ╲ kid │   :Akhil  :Brianna   :Carlos    :Dinah   :Edward
───────────────┼─────────────────────────────────────────────────
:PBJ           │      0.0       0.0       0.0       1.0       0.0
:Turkey        │      1.0       0.0       0.0       0.0       0.0
:Ham           │      0.0       0.0       1.0       0.0       0.0
:Veggie        │      0.0       1.0       0.0       0.0       0.0
:Tuna          │      0.0       0.0       0.0       0.0       1.0

In [7]:
preferences

5×5 Named Matrix{Int64}
sandwich ╲ kid │   :Akhil  :Brianna   :Carlos    :Dinah   :Edward
───────────────┼─────────────────────────────────────────────────
:PBJ           │        2         9         1         9         6
:Turkey        │        9         0         5         5         8
:Ham           │        7         0        10         2         6
:Veggie        │        0         9         8         1         7
:Tuna          │        4         1         4         0         4

In [4]:
x

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, [:PBJ, :Turkey, :Ham, :Veggie, :Tuna]
    Dimension 2, [:Akhil, :Brianna, :Carlos, :Dinah, :Edward]
And data, a 5×5 Matrix{VariableRef}:
 x[PBJ,Akhil]     x[PBJ,Brianna]     …  x[PBJ,Dinah]     x[PBJ,Edward]
 x[Turkey,Akhil]  x[Turkey,Brianna]     x[Turkey,Dinah]  x[Turkey,Edward]
 x[Ham,Akhil]     x[Ham,Brianna]        x[Ham,Dinah]     x[Ham,Edward]
 x[Veggie,Akhil]  x[Veggie,Brianna]     x[Veggie,Dinah]  x[Veggie,Edward]
 x[Tuna,Akhil]    x[Tuna,Brianna]       x[Tuna,Dinah]    x[Tuna,Edward]

In [5]:
a

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, [:Akhil, :Brianna, :Carlos, :Dinah, :Edward]
And data, a 5-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 a[Akhil] : x[PBJ,Akhil] + x[Turkey,Akhil] + x[Ham,Akhil] + x[Veggie,Akhil] + x[Tuna,Akhil] == 1.0
 a[Brianna] : x[PBJ,Brianna] + x[Turkey,Brianna] + x[Ham,Brianna] + x[Veggie,Brianna] + x[Tuna,Brianna] == 1.0
 a[Carlos] : x[PBJ,Carlos] + x[Turkey,Carlos] + x[Ham,Carlos] + x[Veggie,Carlos] + x[Tuna,Carlos] == 1.0
 a[Dinah] : x[PBJ,Dinah] + x[Turkey,Dinah] + x[Ham,Dinah] + x[Veggie,Dinah] + x[Tuna,Dinah] == 1.0
 a[Edward] : x[PBJ,Edward] + x[Turkey,Edward] + x[Ham,Edward] + x[Veggie,Edward] + x[Tuna,Edward] == 1.0

In [6]:
b

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, [:PBJ, :Turkey, :Ham, :Veggie, :Tuna]
And data, a 5-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 b[PBJ] : x[PBJ,Akhil] + x[PBJ,Brianna] + x[PBJ,Carlos] + x[PBJ,Dinah] + x[PBJ,Edward] == 1.0
 b[Turkey] : x[Turkey,Akhil] + x[Turkey,Brianna] + x[Turkey,Carlos] + x[Turkey,Dinah] + x[Turkey,Edward] == 1.0
 b[Ham] : x[Ham,Akhil] + x[Ham,Brianna] + x[Ham,Carlos] + x[Ham,Dinah] + x[Ham,Edward] == 1.0
 b[Veggie] : x[Veggie,Akhil] + x[Veggie,Brianna] + x[Veggie,Carlos] + x[Veggie,Dinah] + x[Veggie,Edward] == 1.0
 b[Tuna] : x[Tuna,Akhil] + x[Tuna,Brianna] + x[Tuna,Carlos] + x[Tuna,Dinah] + x[Tuna,Edward] == 1.0

Notice that even though we didn't tell the variable it had to be binary, it is in the solution! Hmm....

Interesting side-note: Being near the middle on all the rankings caused Edward to end up with his least-favorite sandwich! Let's see what happens if he changes his ratings to be more extreme:

In [8]:
prefs = [ 2 9 1 9 6
        9 0 5 5 9
        7 0 10 2 6
        0 9 8 1 7
        4 1 4 0 3]

preferences = NamedArray( prefs, (sandwiches,kids), ("sandwich","kid"))

m = Model(Clp.Optimizer)

# NOTE: This is just a normal variable! We don't need to specify "binary"
# in this case. We'll see why soon.
@variable(m, x[sandwiches,kids] >= 0)

# each kid gets exactly 1 sandwich
@constraint(m, a[j in kids], sum(x[i,j] for i in sandwiches) == 1 )

# each sandwich gets assigned to one kid
@constraint(m, b[i in sandwiches], sum(x[i,j] for j in kids) == 1 )

@objective(m, Max, sum( x[i,j]*preferences[i,j] for i in sandwiches, j in kids ) )

optimize!(m)

assignment = NamedArray( [ (value(x[i,j])) for i in sandwiches, j in kids ], (sandwiches, kids), ("sandwich","kid"))

Coin0506I Presolve 10 (0) rows, 25 (0) columns and 50 (0) elements
Clp0006I 0  Obj 0 Primal inf 9.999999 (10) Dual inf 117 (21)
Clp0006I 7  Obj 41
Clp0000I Optimal - objective value 41
Clp0032I Optimal objective 41 - 7 iterations time 0.002


5×5 Named Matrix{Float64}
sandwich ╲ kid │   :Akhil  :Brianna   :Carlos    :Dinah   :Edward
───────────────┼─────────────────────────────────────────────────
:PBJ           │      0.0       0.0       0.0       1.0       0.0
:Turkey        │      0.0       0.0       0.0       0.0       1.0
:Ham           │      0.0       0.0       1.0       0.0       0.0
:Veggie        │      0.0       1.0       0.0       0.0       0.0
:Tuna          │      1.0       0.0       0.0       0.0       0.0